In [12]:
import sqlite3
import pandas as pd
import numpy as np
from faker import Faker
from random import choices, randint

# Initialize Faker
fake = Faker()

# Predefined genre-based title templates
genre_titles = {
    'Fiction': ['The Lost Horizon', 'Echoes of Eternity', 'Shadows of the Past', 'A Distant Journey', 'Whispers in the Wind'],
    'Mystery': ['The Hidden Key', 'Murder on the Midnight Express', 'The Silent Witness', 'The Enigma Code', 'Undercover Shadows'],
    'Fantasy': ['The Dragon’s Call', 'The Wizard’s Legacy', 'Chronicles of Eldoria', 'The Crystal Quest', 'The Sorcerer’s Stone'],
    'Science Fiction': ['Galactic Wars', 'The AI Prophecy', 'Beyond the Stars', 'The Quantum Paradox', 'Terraformers'],
    'Romance': ['A Love Unforeseen', 'Hearts Entwined', 'The Paris Affair', 'Whispered Promises', 'A Chance Encounter'],
    'Non-Fiction': ['The Art of Mindfulness', 'Breaking Barriers', 'The Science of Sleep', 'A History of Empires', 'Innovations of Tomorrow'],
    'Biography': ['The Life of a Visionary', 'Memoirs of a Maverick', 'The Untold Story', 'A Journey to Greatness', 'Living Legends'],
    'Thriller': ['The Last Chase', 'Bloodlines', 'The Final Countdown', 'Deadly Pursuit', 'The Betrayal Pact']
}

# Generate Synthetic Book Data
def generate_books(num_books=1000):
    genres = list(genre_titles.keys())
    countries = ['USA', 'UK', 'Canada', 'India', 'Australia', 'Germany', 'France', 'Japan']
    
    books = []
    for _ in range(num_books):
        genre = np.random.choice(genres)
        title = np.random.choice(genre_titles[genre]) + f" {fake.word().capitalize()}"
        author = fake.name()
        year = np.random.randint(1950, 2023)
        # 50% of books are from the USA
        country = 'USA' if np.random.random() < 0.5 else np.random.choice(countries)
        isbn = fake.unique.isbn10()
        books.append([isbn, title, author, genre, year, country])
    
    return pd.DataFrame(books, columns=['ISBN', 'Title', 'Author', 'Genre', 'Year_of_Publication', 'Country'])

# Generate Synthetic User Data
def generate_users(num_users=10000, book_df=None):
    if book_df is None:
        raise ValueError("Book dataset is required to ensure consistency!")
    
    genres = book_df['Genre'].unique()
    countries = book_df['Country'].unique()
    authors = book_df['Author'].unique()
    decades = [(year // 10) * 10 for year in book_df['Year_of_Publication']]
    decades = sorted(list(set(decades)))  # Unique decades
    
    users = {
        'User_ID': range(1, num_users + 1),
        'Age': np.random.randint(15, 80, num_users),
        # 90% Male/Female, 10% Other
        'Gender': np.random.choice(['Male', 'Female', 'Other'], p=[0.45, 0.45, 0.1], size=num_users),
        'Country': np.random.choice(countries, num_users),
        'Prefers_High_Rated_Books': np.random.choice(['Yes', 'No'], num_users),
        'Preferred_Genres': [choices(genres, k=3) for _ in range(num_users)],  # Top 3 genres
        'Preferred_Authors': [choices(authors, k=3) for _ in range(num_users)],  # Top 3 authors
        'Preferred_Decade': np.random.choice(decades, num_users),
        'Likes_Foreign_Books': np.random.choice(['Yes', 'No'], num_users)
    }
    # Join genres and authors as strings for readability
    users['Preferred_Genres'] = [", ".join(prefs) for prefs in users['Preferred_Genres']]
    users['Preferred_Authors'] = [", ".join(prefs) for prefs in users['Preferred_Authors']]
    
    return pd.DataFrame(users)

#Generate Synthetic Ratings Data
def generate_ratings(num_ratings=100000, book_df=None, user_df=None):
    if book_df is None or user_df is None:
        raise ValueError("Book and User datasets are required to generate ratings!")
    
    ratings = {
        'User_ID': np.random.choice(user_df['User_ID'], num_ratings),
        'ISBN': np.random.choice(book_df['ISBN'], num_ratings),
        'Rating': np.random.randint(1, 11, num_ratings)
    }
    
    return pd.DataFrame(ratings)

# Generate the data
num_books = 1000
num_users = 10000
num_ratings = 100000

books_df = generate_books(num_books=num_books)
users_df = generate_users(num_users=num_users, book_df=books_df)
ratings_df = generate_ratings(num_ratings=num_ratings, book_df=books_df, user_df=users_df)

#Save Data to SQLite Database
conn = sqlite3.connect("book_recommendation_system.db")

# Save DataFrames to SQLite tables
books_df.to_sql("Books", conn, if_exists="replace", index=False)
users_df.to_sql("Users", conn, if_exists="replace", index=False)
ratings_df.to_sql("Ratings", conn, if_exists="replace", index=False)

# Save DataFrames to CSV files
books_df.to_csv("synthetic_books.csv", index=False)
users_df.to_csv("synthetic_users.csv", index=False)
ratings_df.to_csv("synthetic_ratings.csv", index=False)

print("Synthetic data has been generated and saved to the SQLite database and CSV files.")

Synthetic data has been generated and saved to the SQLite database and CSV files.


In [13]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Connect to SQLite Database
conn = sqlite3.connect("book_recommendation_system.db")

# Load data from the database
books_df = pd.read_sql_query("SELECT * FROM Books", conn)
users_df = pd.read_sql_query("SELECT * FROM Users", conn)
ratings_df = pd.read_sql_query("SELECT * FROM Ratings", conn)

# Combine metadata for content-based filtering
books_df['metadata'] = (
    books_df['Genre'] + " " +
    books_df['Author'] + " " +
    books_df['Country'] + " " +
    books_df['Year_of_Publication'].astype(str)
)

# TF-IDF Vectorizer for content-based filtering (helps create my sythetic data)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books_df['metadata'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Display user details
def display_user_details(user_id):
    user_preferences = pd.read_sql_query(f"SELECT * FROM Users WHERE User_ID = {user_id}", conn).iloc[0]
    print("User Details:")
    print(f"User ID: {user_preferences['User_ID']}")
    print(f"Name: User_{user_preferences['User_ID']}")  # Simulated name
    print(f"Age: {user_preferences['Age']}")
    print(f"Gender: {user_preferences['Gender']}")
    print(f"Country: {user_preferences['Country']}")
    print(f"Prefers High Rated Books: {user_preferences['Prefers_High_Rated_Books']}")
    print(f"Preferred Genres: {user_preferences['Preferred_Genres']}")
    print(f"Preferred Authors: {user_preferences['Preferred_Authors']}")
    print(f"Preferred Decade: {user_preferences['Preferred_Decade']}")
    print(f"Likes Foreign Books: {user_preferences['Likes_Foreign_Books']}")
    print("-" * 50)

# Content-based recommendation function
def recommend_books_content(user_id, top_n=5):
    user_preferences = pd.read_sql_query(f"SELECT * FROM Users WHERE User_ID = {user_id}", conn).iloc[0]
    preferred_genres = user_preferences['Preferred_Genres'].split(", ")
    preferred_authors = user_preferences['Preferred_Authors'].split(", ")
    preferred_decade = user_preferences['Preferred_Decade']
    likes_foreign = user_preferences['Likes_Foreign_Books'] == "Yes"
    user_country = user_preferences['Country']

    # Apply strict filters
    filtered_books = books_df[
        (books_df['Genre'].isin(preferred_genres)) &
        (books_df['Author'].isin(preferred_authors)) &
        (books_df['Year_of_Publication'] // 10 * 10 == preferred_decade)
    ]
    if not likes_foreign:
        filtered_books = filtered_books[filtered_books['Country'] == user_country]

    # Relax filters if no matches
    if filtered_books.empty:
        print("No strict matches found. Relaxing filters...")
        filtered_books = books_df[
            (books_df['Genre'].isin(preferred_genres)) |
            (books_df['Author'].isin(preferred_authors)) |
            (books_df['Year_of_Publication'] // 10 * 10 == preferred_decade)
        ]
        if not likes_foreign:
            filtered_books = filtered_books[filtered_books['Country'] == user_country]

    # Fallback to all books if still no matches
    if filtered_books.empty:
        print("No matches found even after relaxing filters. Using all books.")
        filtered_books = books_df

    # Compute similarity for filtered books
    indices = filtered_books.index
    user_profile = cosine_sim[indices].mean(axis=0)
    top_indices = np.argsort(user_profile)[::-1][:top_n]
    return books_df.iloc[top_indices][['Title', 'Author', 'Genre', 'Year_of_Publication', 'Country']]

# Rating-based recommendation function
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings_df[['User_ID', 'ISBN', 'Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
model = SVD()
model.fit(trainset)


def recommend_books_rating(user_id, top_n=5):
    user_books = ratings_df[ratings_df['User_ID'] == user_id]['ISBN'].unique()
    all_books = books_df['ISBN']
    non_rated_books = [book for book in all_books if book not in user_books]
    pred_ratings = [(book, model.predict(user_id, book).est) for book in non_rated_books]
    pred_ratings = sorted(pred_ratings, key=lambda x: x[1], reverse=True)[:top_n]
    return books_df[books_df['ISBN'].isin([x[0] for x in pred_ratings])][['Title', 'Author', 'Genre', 'Year_of_Publication', 'Country']]

# Hybrid recommendation
def recommend_books_hybrid(user_id, top_n=5):
    content_recommendations = recommend_books_content(user_id, top_n)
    rating_recommendations = recommend_books_rating(user_id, top_n)
    if isinstance(content_recommendations, str):
        return rating_recommendations
    hybrid_recommendations = pd.concat([content_recommendations, rating_recommendations])
    hybrid_recommendations = hybrid_recommendations.drop_duplicates(subset='Title').head(top_n)
    return hybrid_recommendations

# Test (choose a user between 1 and 10,000)
user_id = 10000
display_user_details(user_id)

print("\nContent-Based Recommendations:")
print(recommend_books_content(user_id))

print("\nRating Recommendations:")
print(recommend_books_rating(user_id))

print("\nHybrid Recommendations:")
print(recommend_books_hybrid(user_id))


User Details:
User ID: 10000
Name: User_10000
Age: 20
Gender: Other
Country: Canada
Prefers High Rated Books: Yes
Preferred Genres: Thriller, Fantasy, Thriller
Preferred Authors: Natalie Shaw, Jessica Brown, Destiny Stuart DVM
Preferred Decade: 2000
Likes Foreign Books: No
--------------------------------------------------

Content-Based Recommendations:
No strict matches found. Relaxing filters...
                           Title             Author      Genre  \
901          The Last Chase Wear    Michael Johnson   Thriller   
551   A Journey to Greatness Two  Michael Rodriguez  Biography   
597  Memoirs of a Maverick Group       Mary Russell  Biography   
106         Bloodlines Candidate       Jose Johnson   Thriller   
802       The Betrayal Pact Than          Mary Dean   Thriller   

     Year_of_Publication Country  
901                 1970  Canada  
551                 2002  Canada  
597                 2002  Canada  
106                 2015  Canada  
802                 1973  